In [3]:
import pandas as pd
import numpy  as np

#################################################
# Principals
#################################################

P = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
P["cat_cln"] = np.where (P['category'].str.contains('actor') | P['category'].str.contains('actress')
                         ,'actor'
                         ,P['category'])
P["cat_cln"] = np.where (P['category'].str.contains('self') | P['category'].str.contains('archive_footage')
                         ,'self',P['cat_cln'])


P.head(55)

,tconst,ordering,nconst,category,job,characters,cat_cln
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",self
1,tt0000001,2,nm0005690,director,\N,\N,director
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N,cinematographer
3,tt0000002,1,nm0721526,director,\N,\N,director
4,tt0000002,2,nm1335271,composer,\N,\N,composer
5,tt0000003,1,nm0721526,director,\N,\N,director
6,tt0000003,2,nm1770680,producer,producer,\N,producer
7,tt0000003,3,nm1335271,composer,\N,\N,composer
8,tt0000003,4,nm5442200,editor,\N,\N,editor
9,tt0000004,1,nm0721526,director,\N,\N,director


In [5]:
import pandas as pd
import numpy  as np
#################################################
# Titles
#################################################

def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

T = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t'
                 , encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                #, error_bad_lines=False
                #, dtype={'startYear': int }
                 , converters={'startYear':conv,'endYear':conv}                
                ) 
T["flg_doc"]           = np.where (T['genres'].str.contains('Documentary'),1,0)
T["primaryTitle_year"] = T['primaryTitle']  + ' ('+  T['startYear'].map(str)  + ')'
## filter movie 
T = T.loc[T.titleType == 'movie', :]
T.head(10)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year
498,tt0000502,movie,Bohemios,Bohemios,0,1905,0,100,\N,0,Bohemios (1905)
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,0,70,"Action,Adventure,Biography",0,The Story of the Kelly Gang (1906)
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,0,90,Drama,0,The Prodigal Son (1907)
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,0,\N,Drama,0,Robbery Under Arms (1907)
625,tt0000630,movie,Hamlet,Amleto,0,1908,0,\N,Drama,0,Hamlet (1908)
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,0,\N,Drama,0,Don Quijote (1908)
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,0,120,"Adventure,Fantasy",0,The Fairylogue and Radio-Plays (1908)
783,tt0000793,movie,Andreas Hofer,Andreas Hofer,0,1909,0,\N,Drama,0,Andreas Hofer (1909)
804,tt0000814,movie,La bocana de Mar Chica,La bocana de Mar Chica,0,1909,0,\N,\N,0,La bocana de Mar Chica (1909)
828,tt0000838,movie,A Cultura do Cacau,A Cultura do Cacau,0,1909,0,\N,\N,0,A Cultura do Cacau (1909)


In [6]:
var_list = ('tconst','primaryTitle' , 'titleType')
#D = newdf.loc[:,('const','title', 'dat_per', 'title_type')]
D = T.loc[:, var_list]
D.head()
pd.get_dummies(D, columns=['titleType']).head(3)


,tconst,primaryTitle,titleType_movie
498,tt0000502,Bohemios,1
570,tt0000574,The Story of the Kelly Gang,1
587,tt0000591,The Prodigal Son,1


In [7]:
#################################################
# Names
#################################################
N = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
N.head(3)


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0117057,tt0071877,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0056404,tt0049189,tt0057345"


In [8]:
#################################################
# Ratings
#################################################
R = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
R.tail(3)

,tconst,averageRating,numVotes
1235277,tt9916730,8.7,6
1235278,tt9916766,6.7,19
1235279,tt9916778,7.2,34


In [9]:
#################################################
# Join
#################################################
from datetime import datetime
             
imdb = pd.merge(    P
                  , T 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(     N.loc[:, ('nconst','primaryProfession', 'primaryName', 'birthYear', 'deathYear')]
                  , imdb
                  , how = 'inner', left_on="nconst", right_on="nconst"
                 )
imdb = pd.merge(     R.loc[:, ('tconst','averageRating', 'numVotes')]
                  , imdb
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )

imdb["birthYear_cln"] = np.where (imdb["birthYear"].str.contains('N')   
                          , "3999"
                          ,imdb["birthYear"])
imdb["deathYear_cln"] = np.where (imdb["deathYear"].str.contains('N')   
                         , "3999"
                          ,imdb["deathYear"]) 
##imdb["startYear_cln"] = np.where (imdb["startYear"].str.contains('N')   
##                       , "1888"
##                          ,imdb["birthYear_cln"]) 
##imdb["age_crew"]            = int(imdb["birthYear_cln"]) -  int(imdb["startYear_cln"])
imdb["last_refresh"]        = datetime.now().strftime('%Y%m%d%H%M%S')
print(P.shape, imdb.shape)


(49905595, 7) (2471653, 26)


In [10]:
imdb.tail(3)

,tconst,averageRating,numVotes,nconst,primaryProfession,primaryName,birthYear,deathYear,ordering,category,...,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,birthYear_cln,deathYear_cln,last_refresh
2471650,tt9916730,8.7,6,nm4852679,actor,Bhushan Pradhan,\N,\N,4,actor,...,0,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918
2471651,tt9916730,8.7,6,nm6096005,actor,Devadhar Archit,\N,\N,1,actor,...,0,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918
2471652,tt9916730,8.7,6,nm9785908,"assistant_director,editor",Rohita More,\N,\N,9,editor,...,0,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918


In [16]:
#################################################
# add row number forfilmmakers with more than one release per year
#################################################
imdb['RN'] = imdb.sort_values(['nconst', 'tconst'], ascending=[True, False]) \
             .groupby(['nconst', 'startYear']) \
             .cumcount() + 1
imdb.tail(100)

,tconst,averageRating,numVotes,nconst,primaryProfession,primaryName,birthYear,deathYear,ordering,category,...,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,birthYear_cln,deathYear_cln,last_refresh,RN
2471553,tt9915436,7.4,5,nm10537887,writer,Joana Kfuri,\N,\N,3,writer,...,2019,0,70,Documentary,1,Vida em Movimento (2019),3999,3999,20220416132918,1
2471554,tt9915436,7.4,5,nm10537888,producer,Marina Costa Manso,\N,\N,5,producer,...,2019,0,70,Documentary,1,Vida em Movimento (2019),3999,3999,20220416132918,1
2471555,tt9915436,7.4,5,nm3644375,"editor,writer,script_department",Selma Perez,\N,\N,4,writer,...,2019,0,70,Documentary,1,Vida em Movimento (2019),3999,3999,20220416132918,1
2471556,tt9915436,7.4,5,nm4704814,"cinematographer,director,producer",Humberto Bassanelli,\N,\N,6,cinematographer,...,2019,0,70,Documentary,1,Vida em Movimento (2019),3999,3999,20220416132918,1
2471557,tt9915436,7.4,5,nm6119329,"director,writer,miscellaneous",Eduardo Rajabally,\N,\N,2,director,...,2019,0,70,Documentary,1,Vida em Movimento (2019),3999,3999,20220416132918,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471648,tt9916730,8.7,6,nm13233318,"actor,production_manager",Ganesh Vasant Patil,\N,\N,3,actor,...,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918,1
2471649,tt9916730,8.7,6,nm1957275,"cinematographer,camera_department,producer",Suresh Deshmane,\N,\N,8,cinematographer,...,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918,1
2471650,tt9916730,8.7,6,nm4852679,actor,Bhushan Pradhan,\N,\N,4,actor,...,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918,1
2471651,tt9916730,8.7,6,nm6096005,actor,Devadhar Archit,\N,\N,1,actor,...,2017,0,116,\N,0,6 Gunn (2017),3999,3999,20220416132918,1


In [17]:
#################################################
#remove rows not needed 
#################################################

X = imdb[imdb['category']!= "production_designer"] 
#X = X[X['category']!= "archive_footage"]
print(X.shape, imdb.shape)

#X.sort_values(by=['tconst', 'averageRating'], ascending=False)
#################################################
# export to file
#################################################
X.to_csv('data/imdb.csv'  ,encoding='utf-8')

(2445521, 27) (2471653, 27)
